In [3]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [8]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def wait2(s):
    pass
wait = wait1

In [5]:
def call(f, *args):
    s = "(" + ", ".join(map(repr, args)) + ")"
    print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            rv = f(*args)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv

### Global functions and routines

In [6]:
def startbatch(name):
    if call(h.batchrunning):
        call(h.endbatch)
    return call(h.startbatch, name)

def settemp(sp):
    if sp == 0:
        call(h.settemp, 2, 37)
    else:
        call(h.settemp, 0, sp)
        
def endbatch():
    call(h.endbatch)
    
def pid1():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 12)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 29)
    
def pid2():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 13)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26)
    
def pid3():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 14)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26)

In [12]:
h = HelloApp('192.168.1.21')

### Round 2 Characterization Tests

In [10]:
hours = 3600
startbatch("tpid 1.28")
pid1()
settemp(37)
wait(3*hours)
endbatch()
settemp(0)
wait(8*hours)
startbatch("tpid 1.29")
pid2()
settemp(37)
wait(3*hours)
endbatch()
settemp(0)
wait(8*hours)
startbatch("tpid 1.30")
pid3()
settemp(37)
wait(3*hours)
endbatch()

Calling batchrunning()
Calling endbatch()
Calling startbatch('tpid 1.28')
Calling setconfig('Temperature', 'P_Gain_(%/C)', 12)
Calling setconfig('Temperature', 'I_Time_(min)', 29)
Calling settemp(0, 37)
Sleeping 10800 seconds, ending at 21:15:47
Calling endbatch()
Calling settemp(2, 37)
Sleeping 28800 seconds, ending at 05:15:49
Calling batchrunning()
Calling startbatch('tpid 1.29')
Calling setconfig('Temperature', 'P_Gain_(%/C)', 13)
Calling setconfig('Temperature', 'I_Time_(min)', 26)
Calling settemp(0, 37)
Sleeping 10800 seconds, ending at 08:15:55
Calling endbatch()
Calling settemp(2, 37)
Sleeping 28800 seconds, ending at 16:15:58
Calling batchrunning()
Calling startbatch('tpid 1.30')
Calling setconfig('Temperature', 'P_Gain_(%/C)', 14)
Calling setconfig('Temperature', 'I_Time_(min)', 26)
Calling settemp(0, 37)
Sleeping 10800 seconds, ending at 19:16:04
Calling endbatch()


### Round 3 verification routines

In [7]:
_bni = 1
def nb(pref, i=1, n=1):
    bl = call(h.getBatches)
    b = bl.names_to_batches
    s = "%s %d.%d" % (pref, i, n)
    while s in b:
        n += 1
        s = "%s %d.%d" % (pref, i, n)
    startbatch(s)

def tempoff():
    settemp(0)
    
def setbn(i):
    global _bni
    _bni = i

def o2a():
    nb("tpid", _bni)
    settemp(37)
    wait(3*hours)
    endbatch()
    
def cool(hrs=8):
    tempoff()
    wait(hrs*hours)
    
def a2a(end):
    nb("tpid", _bni)
    settemp(end)
    wait(3*hours)
    endbatch()

def verify(pid):
    print("Verifying settings")
    pid()
    o2a()
    a2a(35)
    a2a(37)
    
def pid4():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 13)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 28) 

In [27]:
wait(300)
mgbn(3)
verify(pid3)
cool()
mgbn(4)
verify(pid4)
tempoff()

Sleeping 300 seconds, ending at 04/27/2017 01:17:56 PM
Verifying settings
Calling setconfig('Temperature', 'P_Gain_(%/C)', 14)
Calling setconfig('Temperature', 'I_Time_(min)', 26)
Calling batchrunning()
Calling startbatch('tpid v 3.1')
Calling settemp(0, 37)
Sleeping 10800 seconds, ending at 04/27/2017 04:17:58 PM
Calling endbatch()
Calling batchrunning()
Calling startbatch('tpid v 3.2')
Calling settemp(0, 35)
Sleeping 10800 seconds, ending at 04/27/2017 07:18:01 PM
Calling endbatch()
Calling batchrunning()
Calling startbatch('tpid v 3.3')
Calling settemp(0, 37)
Sleeping 10800 seconds, ending at 04/27/2017 10:18:04 PM
Calling endbatch()
Calling settemp(2, 37)
Sleeping 28800 seconds, ending at 04/28/2017 06:18:06 AM
Verifying settings
Calling setconfig('Temperature', 'P_Gain_(%/C)', 13)
Calling setconfig('Temperature', 'I_Time_(min)', 28)
Calling batchrunning()
Calling startbatch('tpid v 4.1')
Calling settemp(0, 37)
Sleeping 10800 seconds, ending at 04/28/2017 09:18:12 AM
Calling endbat

In [6]:
vol = 2.5
beta = 0
data = []
def add(*args):
    data.append(args)
for p, i in ((14, 26), (13, 28)):
    add(p, i, 0, "Off", "RT", 37, vol, beta)
    add(p, i, 0, "Auto", 37, 35, vol, beta)
    add(p, i, 0, "Auto", 35, 37, vol, beta)
for i, args in enumerate(data, 1):
    print(i, *args) #"%d %d %d %s %s %d %d %d"%args)

1 14 26 0 Off RT 37 2.5 0
2 14 26 0 Auto 37 35 2.5 0
3 14 26 0 Auto 35 37 2.5 0
4 13 28 0 Off RT 37 2.5 0
5 13 28 0 Auto 37 35 2.5 0
6 13 28 0 Auto 35 37 2.5 0


In [20]:
startbatch("do 1.3")
call(h.setmg, 1, .2)

Calling batchrunning()
Calling endbatch()
Calling startbatch('do 1.3')
Calling setmg(1, 0.2)


## Round 4 5/24/17
Temp looked a bit unstable in real life in the bio lab. According to simulation, a much higher gain value doens't actually hurt performance, so trying the test out again. 

In [10]:
def pid5():
    call(h.setconfig, "Temperature", "P_Gain_(%/C)", 50)
    call(h.setconfig, "Temperature", 'I_Time_(min)', 26) 
hours = 3600

In [11]:
h = HelloApp('71.189.82.196:81')
wait = wait2
cool(10)
setbn(0)
verify(pid5)
tempoff()

Calling settemp(2, 37)
Verifying settings
Calling setconfig('Temperature', 'P_Gain_(%/C)', 50)
Calling setconfig('Temperature', 'I_Time_(min)', 26)
Calling getBatches()


KeyError: 'ID'

In [52]:
h.login()
call(h.getBatches)

Calling getBatches()


KeyError: 'ID'

In [35]:
print(h.send_request('?call=getBatches').content.decode())

<?xml version="1.0" encoding="UTF-8" standalone="no" ?><Reply><Result>False</Result><Message>No user associated with the getbatches call
Custom Error Message.vi-&gt;Determine User Permission for Call.vi-&gt;Webservice Command Handler.vi-&gt;PBS_WebServer.vi-&gt;PBS_WebServer.vi.ProxyCaller 5038</Message></Reply>


In [38]:
c = h.send_request('?call=getBatches').content
import lxml.etree
root = lxml.etree.XML(c)

In [39]:
m = root[1]
a = m[0]
for c in a[2:]:
    for e in c:
        if e.tag == "I32":
            if e[0].text == "ID":
                break
    else:
        lxml.etree.dump(c)

In [51]:
for e in root.xpath("//Cluster"):
    print(e[2][0].text)

ID
ID
ID
ID
ID
ID
ID
ID
